In [ ]:
from pathlib import Path
from types import SimpleNamespace

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde
from scipy.interpolate import make_interp_spline, BSpline
from astropy.io import ascii

import dustpy
import dustpy.constants as c

import sys
if '..' not in sys.path:
    sys.path.append('..')
import helper
import plot_helper

# Read the paletton color file
helper.set_paletton_colors()

## Load Tazzari data

In [ ]:
tdir = helper.data_dir / 'Tazzari2021'

tb_fr = ascii.read(tdir / 'Table2_Tazzari2021a.txt', format='ipac').to_pandas(index='Name')
tb_fr = tb_fr[np.abs(tb_fr.wle/1.3 - 1) <0.1][['Ftot', 'R68']]

tb_distances = ascii.read(str(tdir / 'Table1_Tazzari2021b.txt'), format='ipac').to_pandas(index='Name')[['d']]
tb_falpha = ascii.read(str(tdir / 'Table2_Tazzari2021b.txt'), format='ipac').to_pandas(index='Name')[['alpha', 'F3mm']]

# get rid of the ones without alpha
nan = np.isnan(tb_falpha['alpha'])
alpha_tazz = tb_falpha[~nan]['alpha']
f3mm = tb_falpha[~nan]['F3mm']
dist = tb_distances[~nan]['d']

# convert to commont distance and 1mm
f1mm = f3mm / ((1. / 3.)**alpha_tazz)
f1mm_tazz_140pc = f1mm * (dist / 140.)**2.

# store in namespace 
taz = SimpleNamespace()
taz.f1mm = f1mm_tazz_140pc.values
taz.alpha = alpha_tazz.values

## Load Testi data

In [ ]:
tes = SimpleNamespace(**np.load(helper.data_dir / 'Testi2014.npz'))
tes.F_mm *= (tes.d_pc / 140.)**2

obs_a = np.concatenate((taz.alpha, tes.alpha_mm))
obs_f = np.concatenate((taz.f1mm, tes.F_mm))

## Compute Observables from dustpy

In [ ]:
import dustpylib.radtrans.slab as slab

In [ ]:
data_dirs = list(Path('~/DATA/araa_data/lowerres_dipsy6').expanduser().glob('data_*'))

In [ ]:
dustpy_data = {}

for dir in data_dirs:

    sn = SimpleNamespace()
    dustpy_data[dir.name] = sn
    
    data = helper.read_dustpy_data(dir)
    it  = np.abs(data.time - 1e5 * c.year).argmin()

    opac = slab.Opacity('ricci_compact.npz')
    lams = np.array([0.089, 0.31])
    obs = slab.slab.get_all_observables(data, opac, lams)
    sn.alpha089_310 = -np.log(obs.flux_t[:, 0]/obs.flux_t[:, 1]) / np.log(lams[0]/lams[1])

    obs2 = slab.slab.get_all_observables(data, opac, [0.1])
    sn.f1mm = obs2.flux_t
    sn.reff = obs2.rf
    sn.time = data.time

## Compute combined Testi-Tazzari KDE

In [ ]:
# compute the KDE
xi = np.log10(obs_f)
yi = obs_a
kde = gaussian_kde(np.array([xi, yi]))

# evaluat the KDE on a grid
x = np.linspace(np.log10(8), 3, 200)
y = np.linspace(1, 4, 100)
X, Y = np.meshgrid(x, y, indexing='ij')
dens = kde(np.array([X.ravel(), Y.ravel()])).reshape(X.shape)

# find levels containing a fraction `lev` of all density
dens_sort = np.sort(dens.ravel())
cum = np.cumsum(dens_sort)
cum /= cum[-1]
levels = [dens_sort.ravel()[np.abs(cum-(1-lev)).argmin()] for lev in [0.68, 0.25]]

## Read Andrews Data

In [ ]:
sa = plot_helper.get_seans_data('../data/Andrews2018/ALLDISKS.summary.dat')

## Compute KDE of the SLR

In [ ]:
# compute the KDE
xi = np.log10(sa.reff[sa.mask])
yi = np.log10(sa.flux[sa.mask])
kde = gaussian_kde(np.array([xi, yi]))

# evaluat the KDE on a grid
x2 = np.linspace(xi.min() - 0.1, xi.max() + 0.1, 200)
y2 = np.linspace(yi.min() - 0.1, yi.max() + 0.1, 150)
X2, Y2 = np.meshgrid(x2, y2, indexing='ij')
dens2 = kde(np.array([X2.ravel(), Y2.ravel()])).reshape(X2.shape)

# find levels containing a fraction `lev` of all density
dens2_sort = np.sort(dens2.ravel())
cum2 = np.cumsum(dens2_sort)
cum2 /= cum2[-1]
levels2 = [dens2_sort.ravel()[np.abs(cum2 - (1 - lev)).argmin()] for lev in [0.68, 0.25]]

# Create a combined figure

In [ ]:
f, axs = plt.subplots(1, 2, figsize=(13, 4), dpi=100)


# ###########################
# first plot: FLUX vs ALPHA #
# ###########################
ax = axs[0]
ax.set_xscale('log')

# plot the mesh and contours
ax.pcolormesh(10.**x, y, dens.T, cmap='Blues', rasterized=True)
cs = ax.contour(10.**x, y, dens.T, np.sort(levels), colors='k', linestyles=['--', '-'])
cl = ax.clabel(cs, fontsize=8, inline=1, inline_spacing=0.1, manual=((100, 2.), (3e2, 1.5)),
               fmt=lambda val: f'{1-np.interp(val, dens_sort, cum):.0%}')

# add obs. scatter points
ax.scatter(tes.F_mm, tes.alpha_mm, c='0.25', s=5, label='Testi et al. 2014')
ax.scatter(taz.f1mm, taz.alpha, c='0.75', s=5, label='Tazzari et al. 2021')


# add dustpy simulations

for key in dustpy_data.keys():
    d = dustpy_data[key]
    if 'nobump' in key:
        label = 'no trap'
    else:
        label = 'efficient trap'

    it = np.abs(d.time - 1e5 * c.year).argmin()
    helper.plot_time_path(1e3 * d.f1mm[it:], d.alpha089_310[it:], d.time[it:] / (1e6 * c.year),
                          snaps=[0.1, 0.5, 1, 2, 3], eps=0, tlog=True, xlog=True,
                          ax=ax, k=2, spline=True, label=label)

# adjust figure
ax.set_xlim(10.**x[[0, -1]])
ax.set_ylim(y[[0, -1]])
ax.set_xlabel(r'$\mathbf{F_\nu^{1mm}}$ [mJy]')
ax.set_ylabel(r'$\mathbf{\alpha_{0.89-3mm}}$')
leg = ax.legend(edgecolor='k', fontsize='small')
leg.get_frame().set_alpha(0.5)

ax.set_facecolor('none')


# ###################
# second plot: SLR  #
# ###################

ax = axs[1]
ax.loglog()
# plot the mesh and contours
ax.pcolormesh(10.**x2, 10.**y2, dens2.T, cmap='Blues', rasterized=True)
cs = ax.contour(10.**x2, 10.**y2, dens2.T, np.sort(levels2), colors='k', linestyles=['--', '-'])
cl = ax.clabel(cs, fontsize=8, inline=1, inline_spacing=15, manual=((80, 0.4), (70, 1)),
               fmt=lambda val: f'{1-np.interp(val, dens2_sort, cum2):.0%}')

###### overplot the dustpy data ######

for key in dustpy_data.keys():

    d = dustpy_data[key]
    if 'nobump' in key:
        label = 'no trap'
    else:
        label = 'efficient trap'

    it = np.abs(d.time - 1e5 * c.year).argmin()
    helper.plot_time_path(d.reff[it:] / c.au, d.f1mm[it:], d.time[it:] / (1e6 * c.year),
                          snaps=[0.1, 0.5, 1, 2, 3], eps=0, tlog=True, xlog=True,
                          ax=ax, k=2, spline=True, label=label)


###### reproducing seans plot ######

# the good
ma = sa.mask
ax.errorbar(sa.reff[ma], sa.flux[ma], xerr=[sa.reff_l[ma], sa.reff_h[ma]], yerr=[sa.flux_l[ma], sa.flux_h[ma]],
                xuplims=sa.reff_f[ma],
                uplims=sa.flux_f[ma],
                marker='.', markersize=10, ecolor='k', mfc='k', mec='none',
                elinewidth=0.5, linestyle='none', label='Andrews et al. (2018)')
    
# the bad: we set the lower limit length to 0.1 times the value
ma = ~sa.mask
ax.errorbar(sa.reff[ma], sa.flux[ma], xerr=[sa.reff[ma]*0.1, sa.reff_h[ma]], yerr=[sa.flux_l[ma], sa.flux_h[ma]],
                xuplims=sa.reff_f[ma],
                uplims=sa.flux_f[ma],
                marker='.', markersize=10, ecolor='0.5', mfc='0.5', mec='none',
                elinewidth=0.5, linestyle='none', label='Andrews at al. (2018)\nupper limits')


####### plot the correlation ######

_x = np.array(ax.get_xlim())
ax.plot(_x, 10.**((np.log10(_x) - 2.1 + 0.0) / 0.49), '-', c='0.5', label='Size-Luminosity relation\n(Tripathi et al. 2017)')
ax.plot(_x, 10.**((np.log10(_x) - 2.1 + 0.2) / 0.49), '--', c='0.5')
ax.plot(_x, 10.**((np.log10(_x) - 2.1 - 0.2) / 0.49), '--', c='0.5')

# plot tweaks
ax.legend(fontsize='small')
ax.set_ylabel(r'${\rm log}$' + ' ' + r'$L_{\rm mm}$' + ' / ' + r'${\rm Jy}$', fontsize=8)
ax.set_xlabel(r'${\rm log}$' + ' ' + r'$R_{\rm eff}$' + ' / ' + r'${\rm au}$', fontsize=8)

###### other figure settings #####

ax.set_xlim(10.**x2[[0, -1]])
ax.set_ylim(10.**y2[[0, -1]])

ax.set_xlabel(r'$\mathbf{r_\mathrm{eff}}$ [au]')
ax.set_ylabel(r'$\mathbf{L_{mm}}$ [Jy]')
#ax.legend().get_frame().set_facecolor(0);
ax.set_facecolor('none')

helper.apply_araa_style(ax)
f.savefig(helper.output_dir / 'BirnstielFig10.pdf', transparent=True, bbox_inches='tight')